In [ ]:
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('properties_data3.csv')
# df.head()

In [ ]:
df_houses = df[df['Type'] == 'house']
df_apartments = df[df['Type'] == 'apartment']

In [ ]:
y = df_houses['Price']


In [ ]:
# Split the data into training and testing sets

In [ ]:
# Create a logistic regression model

In [ ]:
# Train the model

In [ ]:
# Make predictions on the test set

In [ ]:
# Evaluate the model